In [1]:
import requests
import json
import time
import threading
from datetime import datetime, timedelta

In [3]:
states = json.loads(requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/states").text)['states']

In [4]:
states_info = {}
districts_rev = {}
for state in states:
    dists = json.loads(requests.get(f"https://cdn-api.co-vin.in/api/v2/admin/location/districts/{state['state_id']}").text)['districts']
    districts = {}
    for d in dists:
        districts[d['district_id']] = d['district_name']
        districts_rev[d['district_name'].lower()] = d['district_id']
    states_info[state['state_id']] = {'name':state['state_name'], 'districts':districts}

In [5]:
states_info

{1: {'name': 'Andaman and Nicobar Islands',
  'districts': {3: 'Nicobar',
   1: 'North and Middle Andaman',
   2: 'South Andaman'}},
 2: {'name': 'Andhra Pradesh',
  'districts': {9: 'Anantapur',
   10: 'Chittoor',
   11: 'East Godavari',
   5: 'Guntur',
   4: 'Krishna',
   7: 'Kurnool',
   12: 'Prakasam',
   13: 'Sri Potti Sriramulu Nellore',
   14: 'Srikakulam',
   8: 'Visakhapatnam',
   15: 'Vizianagaram',
   16: 'West Godavari',
   6: 'YSR District, Kadapa (Cuddapah)'}},
 3: {'name': 'Arunachal Pradesh',
  'districts': {22: 'Anjaw',
   20: 'Changlang',
   25: 'Dibang Valley',
   23: 'East Kameng',
   42: 'East Siang',
   17: 'Itanagar Capital Complex',
   24: 'Kamle',
   27: 'Kra Daadi',
   21: 'Kurung Kumey',
   33: 'Lepa Rada',
   29: 'Lohit',
   40: 'Longding',
   31: 'Lower Dibang Valley',
   18: 'Lower Siang',
   32: 'Lower Subansiri',
   36: 'Namsai',
   19: 'Pakke Kessang',
   39: 'Papum Pare',
   35: 'Shi Yomi',
   37: 'Siang',
   30: 'Tawang',
   26: 'Tirap',
   34: 'Upper

In [6]:
districts_rev

{'nicobar': 3,
 'north and middle andaman': 1,
 'south andaman': 2,
 'anantapur': 9,
 'chittoor': 10,
 'east godavari': 11,
 'guntur': 5,
 'krishna': 4,
 'kurnool': 7,
 'prakasam': 12,
 'sri potti sriramulu nellore': 13,
 'srikakulam': 14,
 'visakhapatnam': 8,
 'vizianagaram': 15,
 'west godavari': 16,
 'ysr district, kadapa (cuddapah)': 6,
 'anjaw': 22,
 'changlang': 20,
 'dibang valley': 25,
 'east kameng': 23,
 'east siang': 42,
 'itanagar capital complex': 17,
 'kamle': 24,
 'kra daadi': 27,
 'kurung kumey': 21,
 'lepa rada': 33,
 'lohit': 29,
 'longding': 40,
 'lower dibang valley': 31,
 'lower siang': 18,
 'lower subansiri': 32,
 'namsai': 36,
 'pakke kessang': 19,
 'papum pare': 39,
 'shi yomi': 35,
 'siang': 37,
 'tawang': 30,
 'tirap': 26,
 'upper siang': 34,
 'upper subansiri': 41,
 'west kameng': 28,
 'west siang': 38,
 'baksa': 46,
 'barpeta': 47,
 'biswanath': 765,
 'bongaigaon': 57,
 'cachar': 66,
 'charaideo': 766,
 'chirang': 58,
 'darrang': 48,
 'dhemaji': 62,
 'dhubri

In [7]:
states_rev = {s['state_name'].lower(): s['state_id'] for s in states}
states_rev

{'andaman and nicobar islands': 1,
 'andhra pradesh': 2,
 'arunachal pradesh': 3,
 'assam': 4,
 'bihar': 5,
 'chandigarh': 6,
 'chhattisgarh': 7,
 'dadra and nagar haveli': 8,
 'daman and diu': 37,
 'delhi': 9,
 'goa': 10,
 'gujarat': 11,
 'haryana': 12,
 'himachal pradesh': 13,
 'jammu and kashmir': 14,
 'jharkhand': 15,
 'karnataka': 16,
 'kerala': 17,
 'ladakh': 18,
 'lakshadweep': 19,
 'madhya pradesh': 20,
 'maharashtra': 21,
 'manipur': 22,
 'meghalaya': 23,
 'mizoram': 24,
 'nagaland': 25,
 'odisha': 26,
 'puducherry': 27,
 'punjab': 28,
 'rajasthan': 29,
 'sikkim': 30,
 'tamil nadu': 31,
 'telangana': 32,
 'tripura': 33,
 'uttar pradesh': 34,
 'uttarakhand': 35,
 'west bengal': 36}

In [31]:
def find_appointments(district_ids, date, search_for_18_plus=True):
    result = []
    for district_id in district_ids:
        print(district_id)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 
        url = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/calendarByDistrict?district_id={}&date={}".format(district_id, date)
        public_url = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}".format(district_id, date)
        response = requests.get(url)
        if response.status_code != 200:
            print("Non-public URL didnt work")
            response = requests.get(public_url, headers=headers)
        if response.status_code != 200:
            print("Public URL also didn't work", headers=headers)
            continue
        appointments = json.loads(response.text)
        centers = appointments['centers']
        for center in centers:
            for session in center['sessions']:
                if session['available_capacity'] == 0:
                    continue
                if search_for_18_plus and session['min_age_limit'] > 18:
                    continue
                result.append("{} - {} - {} - {}".format(session['date'], center['name'],
                                                         center['district_name'], center['pincode']))
    return result

In [32]:
def valid_state(state):
    return state.lower() in states_rev.keys()

def valid_district(district):
    return district.lower() in districts_rev.keys()

In [33]:
from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove, Update
from telegram.ext import (
    Updater,
    CommandHandler,
    MessageHandler,
    Filters,
    ConversationHandler,
    CallbackContext,
)


In [34]:
STATE, DISTRICT, AGE = range(3)

In [35]:
def remove_job_if_exists(name: str, context: CallbackContext) -> bool:
    current_jobs = context.job_queue.get_jobs_by_name(name)
    if not current_jobs:
        return False
    for job in current_jobs:
        job.schedule_removal()
    return True

In [36]:
def start(update: Update, context: CallbackContext) -> int:
    remove_job_if_exists(str(update.message.chat_id), context)
    update.message.reply_text(
        'Hi! Im Cowin bot. I will help you book covid vaccination appointment near you. '
        'Send /stop anytime to stop talking to me.\n\n'
        'Which state are you in?\n'
        'Possible values:\n'+'\n'.join(states_rev.keys())
    )

    return STATE

In [37]:
def state(update: Update, context: CallbackContext) -> int:
    user = update.message.from_user
    chosen_state = update.message.text
    if not valid_state(chosen_state):
        update.message.reply_text('Please enter a valid state')
        return STATE

    state_id = states_rev[chosen_state.lower()]
    context.user_data["state"] = state_id
    possible_districts = list(states_info[state_id]['districts'].values())
    update.message.reply_text('Which district(s) you want to search for? Enter comma separated values.\nPossible values:\n{}'.format("\n".join(possible_districts)))
    return DISTRICT

In [38]:
def district(update: Update, context: CallbackContext) -> int:
    user = update.message.from_user
    chosen_districts = update.message.text.split(",")
    chosen_districts = [d.strip() for d in chosen_districts]
    
    if not all(valid_district(district) for district in chosen_districts):
        update.message.reply_text('Please enter valid district(s).')
        return DISTRICT
    
    reply_keyboard = [['Yes', 'No']]
    district_ids = [districts_rev[d.lower()] for d in chosen_districts]
    context.user_data["districts"] = district_ids
    update.message.reply_text('Do you want to search only for slots available for 18-45 age group?',
                              reply_markup=ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True))
    return AGE

In [39]:
def callback_alarm(context: CallbackContext):
    user_data = context.job.context 
    appointments = find_appointments(user_data['districts'], datetime.today().strftime('%d-%m-%Y'), 
                                     user_data['only_18_plus'])
    if len(appointments) > 0:
        context.bot.send_message(chat_id=user_data['chat_id'], 
                                 text="The slots are available!! Centers -\n{}\nSend /stop anytime to stop talking to me.\n\n".format("\n".join(appointments[:20])))

In [40]:
def age(update: Update, context: CallbackContext) -> int:
    user = update.message.from_user
    only_18_plus = update.message.text.lower() == 'yes'
    context.user_data["only_18_plus"] = only_18_plus
    chat_id = update.message.chat_id
    context.user_data.update({'chat_id':chat_id})
    remove_job_if_exists(str(chat_id), context)
    context.job_queue.run_repeating(callback_alarm, 60, context=context.user_data, name=str(chat_id))
    update.message.reply_text('I will update you if any slots open up! Till then, take care and stay home if possible.', 
                              reply_markup=ReplyKeyboardRemove())
    return AGE

In [41]:
def stop(update: Update, context: CallbackContext) -> int:
    user = update.message.from_user
    update.message.reply_text(
        'Bye {}! Take care and stay home if possible.\n\nSend /start anytime to start talking to me.'.format(user.first_name), 
        reply_markup=ReplyKeyboardRemove()
    )
    remove_job_if_exists(str(update.message.chat_id), context)
    return ConversationHandler.END

In [ ]:
updater = Updater(token='1794674455:AAGjQarqlo_WFqopHyqVL-C7xCIkhjV2lAs')

dispatcher = updater.dispatcher

conv_handler = ConversationHandler(
    entry_points=[CommandHandler('start', start)],
    states={
        STATE: [MessageHandler(Filters.text & ~Filters.command, state)],
        DISTRICT: [MessageHandler(Filters.text & ~Filters.command, district)],
        AGE: [MessageHandler(Filters.regex('^(Yes|No)$'), age)]
    },
    fallbacks=[CommandHandler('stop', stop)],
)

dispatcher.add_handler(conv_handler)

updater.start_polling()

updater.idle()

506
Non-public URL didnt work
505
Non-public URL didnt work
506
Non-public URL didnt work
505
Non-public URL didnt work
506
Non-public URL didnt work
505
Non-public URL didnt work
506
Non-public URL didnt work
505
Non-public URL didnt work
506
Non-public URL didnt work
505
Non-public URL didnt work
506
Non-public URL didnt work
505
Non-public URL didnt work
506
Non-public URL didnt work
505
Non-public URL didnt work
506
Non-public URL didnt work
505
Non-public URL didnt work
506
Non-public URL didnt work
505
Non-public URL didnt work
506
Non-public URL didnt work
505
Non-public URL didnt work
506
Non-public URL didnt work
505
Non-public URL didnt work
506
Non-public URL didnt work
505
Non-public URL didnt work
506
Non-public URL didnt work


In [20]:
find_appointments([506], '05-05-2021', False)

https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id=506&date=05-05-2021


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [30]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like
requests.get("https://cdn-api.co-vin.in/api/v2/appointment/sessions/calendarByDistrict?district_id=506&date=05-05-2021", headers=headers).status_code

401